# Random Forests from Scratch
- Step-by-step tutorial [here](https://machinelearningmastery.com/implement-random-forest-scratch-python/)
- Video walkthrough [here](https://www.youtube.com/watch?v=QHOazyP-YlM)
- Podcast [here](https://podtail.com/en/podcast/data-skeptic/mini-decision-tree-learning/) and [here](https://www.acast.com/dataskeptic/-mini-random-forest)

Ok, you are looking at a new restaurant and wondering if you should go to it. You have a "data set" \[price, location, food quality, type of food, wasGoodChoice\] of all the past restaurants you went to... how are you going to make your decision? Go to the restaurant or not??
1. Get an opinion from a friend who has also been to all the same restaurant with you and cares about everything exactly the same as you. They will give you thier input and thats how you decide. Whats wrong with this?
    - This is just too biased on the decision of one friend, what about getting more friends to chip in?
    - If that person is have a bad day they may decide widely different then if they were having a good day. Your one friend has "high variability".
    - This is analogous to a single decision tree.
2. So now you have 10 friends that all care about the same things as you, but they each only went to a few of the restaurants that you have. You ask them all to pitch in thier opinion.
    - They all have slighly different backgrounds so they may give you a different answer but they all care about the exact same things, so they will most likely have vary similar opinions.
    - You mitigated the "high variability" of only one friend, but they're decision making process is going to have "high correlation" with eachother.
    - This is analogous to "bagging".
3. So you take 10 friends who all care about different features of the restaurant and each only went to a few of the restaurants that you have. i.e. Joe cares about price and location and could care less about food quality and type of food. You ask them all to pitch in thier opinion.
    - This time you've found a happy middle between variability (one friend) and correlation (friends with same taste)
    - Each friend is going to pick slighly different based on different criteria and you get to take all their input to make a final decision.
    - This happy medium is called random forest algorithm :)

##### The next four code boxes are copied nearly directly from the tutorial linked at the beginning. Run them all through before continuing.

In [12]:
# Random Forest Algorithm on Sonar Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt

In [13]:
# First, the decision tree algorithm...
# Select the best split point for a dataset
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    # features_subset = random.sample(range(1, len(dataset[0])-1), n_features)
    features_subset = list()
    while len(features_subset) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features_subset.append(index)
    # go through each feature in the subset of features
    for feature_index in features_subset:
        # go through each data sample
        for row in dataset:
            # split the data set into two groups based on a canidate split threshold
            groups = test_split(index, row[feature_index], dataset)
            # access the success of that split to correctly
            # place the data in its correct categories
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = feature_index, row[feature_index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [14]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

# Select the best split point for a dataset
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

# Create a random subsample from the dataset with replacement
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

# Make a prediction with a list of bagged trees
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

# Random Forest Algorithm
def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)

In [27]:
# load and prepare data
sonar_filename = 'sonar.all-data.csv'
sonar_dataset = load_csv(sonar_filename)

In [16]:
# Test the random forest algorithm
seed(2)
# convert string attributes to integers
for i in range(0, len(sonar_dataset[0])-1):
    str_column_to_float(sonar_dataset, i)
# convert class column to integers
str_column_to_int(sonar_dataset, len(sonar_dataset[0])-1)
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(sonar_dataset[0])-1))
for n_trees in [1, 5, 10]:
    scores = evaluate_algorithm(sonar_dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print('Trees: %d' % n_trees)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Trees: 1
Scores: [56.09756097560976, 63.41463414634146, 60.97560975609756, 58.536585365853654, 73.17073170731707]
Mean Accuracy: 62.439%
Trees: 5
Scores: [70.73170731707317, 58.536585365853654, 85.36585365853658, 75.60975609756098, 63.41463414634146]
Mean Accuracy: 70.732%
Trees: 10
Scores: [82.92682926829268, 75.60975609756098, 97.5609756097561, 80.48780487804879, 68.29268292682927]
Mean Accuracy: 80.976%


OK, now lets go through it. How is a tree constructed?
- Each node uses a "gini cost function" to determine which threshold to pick for that node.
- The goal is to find ideal threashold that puts every (as many as possible) data point from class A in the right-hand bucket and every other point in the class B in the left-hand bucket.
- Gini value of 0 is a "pure bucket", meaning in the LH (or RH) bucket, all the points are from class A or all from class B.
- Gini value of .5 is a 50/50 bucket, meaning in the LH (or RH) bucket, half the points are from class A and half class B.

### Visualizing a Decision Tree
#### You will need to install the following:
- pip3 install scipy
- brew install xdot
    - or you can:
        - brew install graphviz
        - dot -Tpng DocName.dot -o DocName.png


In [17]:
from sklearn.datasets import load_iris
from sklearn import tree
clf = tree.DecisionTreeClassifier()

In [18]:
# first a dummy example
iris = load_iris()
clf = clf.fit(iris.data, iris.target)
tree.export_graphviz(clf,
    feature_names=["Sepal Length", "Sepal Width", "Petal Length","Petal Width"],
    class_names=["Setosa", "Versicolour", "Virginica"],
    out_file='tree.dot')

In [34]:
# import pandas for the next part
import pandas as pd
import numpy as np

In [56]:
# now with the sonar data set from above
sonar_df = pd.read_csv(sonar_filename, header=None)
sonar_data, sonar_labels = np.split(sonar_df, [60], axis=1)
# here we are converting the labels {"R", "M"} to {0, 1}
# note to self: find better way to make series from dataframe
sonar_labels = pd.factorize(sonar_labels.T.iloc[0])

In [58]:
# more to come ...